In [15]:
import gym

In [16]:
from spinup import sac_pytorch as sac

In [17]:
import torch
import torch.nn as nn
class HEDGING_Actor(nn.Module):

    def __init__(self, obs_dim, act_dim, act_limit):
        super().__init__()
        self.fc1 = nn.Linear(obs_dim, 3)
        self.ac1 = nn.LeakyReLU(0.1)
        self.fc2 = nn.Linear(3, 2)
        self.ac2 = nn.LeakyReLU(0.1)
        self.fc3 = nn.Linear(2, act_dim)

    def forward(self, obs):
        # Return output from network scaled to action space limits.
        x = self.fc1(obs)
        x = self.ac1(x)
        x = self.fc2(x)
        x = self.ac2(x)
        x = self.fc3(x)
        x = torch.clamp(x, -2, 2)
        return x

class HEDGING_QFunction(nn.Module):

    def __init__(self, obs_dim, act_dim):
        super().__init__()
        self.fc1 = nn.Linear(obs_dim + act_dim, 3)
        self.ac1 = nn.LeakyReLU(0.1)
        self.fc2 = nn.Linear(3, 2)
        self.ac2 = nn.LeakyReLU(0.1)
        self.fc3 = nn.Linear(2, act_dim)

    def forward(self, obs, act):
        x = self.fc1(torch.cat([obs, act], dim=-1))
        x = self.ac1(x)
        x = self.fc2(x)
        x = self.ac2(x)
        x = self.fc3(x)
        return torch.squeeze(x, -1) # Critical to ensure q has right shape.

class HEDGING(nn.Module):

    def __init__(self, observation_space, action_space,activation=nn.ReLU):
        super().__init__()

        obs_dim = observation_space.shape[0]
        act_dim = action_space.shape[0]
        act_limit = action_space.high[0]

        # build policy and value functions
        self.pi = HEDGING_Actor(obs_dim, act_dim, act_limit)
        self.q1 = HEDGING_QFunction(obs_dim, act_dim)

    def act(self, obs):
        with torch.no_grad():
            return self.pi(obs).numpy()

In [18]:
sac(env_fn=lambda : gym.make('DHENV:DHENV-v1'),lr = 1e-4)

Logging data to /tmp/experiments/1596717351/progress.txt
Saving config:

{
    "ac_kwargs":	{},
    "actor_critic":	"HEDGING",
    "alpha":	0.2,
    "batch_size":	100,
    "env_fn":	"<function <lambda> at 0x157b7b3b0>",
    "epochs":	100,
    "gamma":	0.99,
    "logger":	{
        "<spinup.utils.logx.EpochLogger object at 0x156dae7d0>":	{
            "epoch_dict":	{},
            "exp_name":	null,
            "first_row":	true,
            "log_current_row":	{},
            "log_headers":	[],
            "output_dir":	"/tmp/experiments/1596717351",
            "output_file":	{
                "<_io.TextIOWrapper name='/tmp/experiments/1596717351/progress.txt' mode='w' encoding='UTF-8'>":	{
                    "mode":	"w"
                }
            }
        }
    },
    "logger_kwargs":	{},
    "lr":	0.0001,
    "max_ep_len":	1000,
    "num_test_episodes":	10,
    "polyak":	0.995,
    "replay_size":	1000000,
    "save_freq":	1,
    "seed":	0,
    "start_steps":	10000,
    "steps_per

AttributeError: 'HEDGING' object has no attribute 'q1'